<a href="https://colab.research.google.com/github/sunny0103/DeepLearning_projects/blob/main/AIconnect_translation/aiconnect_kor_eng_translate_hyperparemetersearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  transformers[sentencepiece] accelerate  sacremoses evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 18.5 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 11.4 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=c78968b2d0a94b1f64f064833892062939a4874a066a70b6d29807d79cdafe00
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [ ]:
!pip install wandb --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: wandb
    Found existing installation: wandb 0.15.5
    Uninstalling wandb-0.15.5:
      Successfully uninstalled wandb-0.15.5


In [ ]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import pandas as pd
import numpy as np
import os
import random
from tqdm import tqdm

from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    pipeline
    )

from datasets import (
    load_dataset,
    DatasetDict,
    load_metric
    )

import warnings
warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [ ]:
!pip install gdown
!gdown --id # data id

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1sx-5J-sP86bUSodQ5Xd8YiHD8L5OlCzi
To: /kaggle/working/train.csv
100%|███████████████████████████████████████| 52.9M/52.9M [00:00<00:00, 134MB/s]


In [ ]:
def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)

seed_everything(42)

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

In [ ]:
batch_name = f'helsinki_nlp_ko_en_{random.randrange(99999999)}'
batch_name

'helsinki_nlp_ko_en_85822412'

In [ ]:
raw_dataset = load_dataset("csv", data_files='train.csv', split='train')
# get 1 percent of data for test hyperparameter search
train_valid = raw_dataset.train_test_split(.01)
# split 20 percent for test 80 percent for train
data_20 = train_valid['test'].train_test_split(.2)
split_datasets = DatasetDict({
    'train': data_20['train'],
    'valid': data_20['test'],
})
split_datasets


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a6751473b96d81b5/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


DatasetDict({
    train: Dataset({
        features: ['sid', '한국어', '영어'],
        num_rows: 1280
    })
    valid: Dataset({
        features: ['sid', '한국어', '영어'],
        num_rows: 320
    })
})

In [ ]:
print("원문: ", split_datasets['train'][0]['한국어'])
print("번역문: ", split_datasets['train'][0]['영어'])

원문:  청화산의 남쪽 사면에는 화강암이 분포하고 있는데, 이 암석은 건설용 재료로 활용된다.
번역문:  Granite is distributed on the southern slope of Cheonghwasan Mountain, and this rock is also used as a material for construction.


In [ ]:
MODEL_NAME = "Helsinki-NLP/opus-mt-ko-en"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
max_input_length = 32
max_target_length = 32

def preprocess_function(examples):
  inputs = [ex for ex in examples['한국어']]
  targets = [ex for ex in examples['영어']]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

  # tokenizer setup for target data
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length= max_target_length, truncation=True)

  model_inputs['labels'] = labels['input_ids']
  return model_inputs

In [ ]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
metric = load_metric("sacrebleu")

In [ ]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # -100은 건너뛴다.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # 단순 후처리
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [ ]:
# confit default and sweep config
NUM_EPOCHS = 3
BATCH_SIZE = 16
LEARNING_RATE = 5e-5
SCHEDULER_TYPE = "linear"

config_defaults ={
    'epochs' : NUM_EPOCHS,
    'batch_size':BATCH_SIZE,
    'learning_rate':LEARNING_RATE,
    'weight_decay':0.0,
    'optimizer' :'AdamW',
    'SchedulerType': SCHEDULER_TYPE
}

sweep_config ={
    'method':'random', #'grid', 'random', 'bayes'
    'name':'sweep',
    'metric':{
        'name':'bleu',
        'goal': 'maximize'
    },
    'parameters':{
        'learning_rate': {'max':1e-1, 'min': 5e-5},
        'weight_decay':  {'values': [0.01, 0.03, 0.1, 0.3, 0.5]},
        'SchedulerType':{'values': ["linear", "cosine", "constant"]}
    },


}


In [ ]:
# import torch,  gc
# gc.collect()
# torch.cuda.empty_cache()
# torch.cuda.memory_summary(device=None, abbreviated=False)

def train():
  with wandb.init(config=config_defaults):
    config = wandb.config

    args = Seq2SeqTrainingArguments(
        output_dir = batch_name,
        evaluation_strategy="epoch",
        learning_rate = config.learning_rate,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=NUM_EPOCHS,
        weight_decay=config.weight_decay,
        lr_scheduler_type = config.SchedulerType,
        gradient_accumulation_steps = 8,
        predict_with_generate=True,
        fp16=True, # 고속화 loose한 정확도
        report_to="wandb",
        # gradient_checkpointing=True, # 메모리 절약 대신 느려짐
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["valid"],
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()

In [ ]:
entity = "solowosophia13"
sweep_id = wandb.sweep(sweep_config, entity= entity, project="AIconnect_ko_en_translation")
wandb.agent(sweep_id, train, count=20)

Create sweep with ID: sa9bqp9j
Sweep URL: https://wandb.ai/solowosophia13/AIconnect_ko_en_translation/sweeps/sa9bqp9j


wandb: Agent Starting Run: plj0f827 with config:
wandb: 	SchedulerType: constant
wandb: 	learning_rate: 0.027266047163388468
wandb: 	weight_decay: 0.1
wandb: Currently logged in as: solowosophia13. Use `wandb login --relogin` to force relogin


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Bleu
1,No log,83.006355,0.000889
2,No log,91.990952,0.001034
3,No log,106.000488,0.000793


eval/bleu,▄█▁
eval/loss,▁▄█
eval/runtime,█▁▂
eval/samples_per_second,▁██
eval/steps_per_second,▁▁▁
train/epoch,▁▅██
train/global_step,▁▅██
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁


...

eval/bleu,▁▁▁
eval/loss,▁▁▁
eval/runtime,█▁▄
eval/samples_per_second,▁█▅
eval/steps_per_second,▁▁▁
train/epoch,▁▅██
train/global_step,▁▅██
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁


wandb: Agent Starting Run: wecq42bv with config:
wandb: 	SchedulerType: cosine
wandb: 	learning_rate: 0.024403682573358176
wandb: 	weight_decay: 0.01


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Bleu
1,No log,65.408493,0.000782
2,No log,65.408493,0.000782
3,No log,65.408493,0.000782


eval/bleu,▁▁▁
eval/loss,▁▁▁
eval/runtime,█▁▁
eval/samples_per_second,▁██
eval/steps_per_second,▁██
train/epoch,▁▅██
train/global_step,▁▅██
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁
train/train_samples_per_second,▁


In [ ]:
wandb.finish()